# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter).
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``.

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [1]:
import torch
import numpy as np

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$.
1. Compute $C = A B$ using matrix-matrix operations and report the time.
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [2]:
# Construct matrics A and B
A = torch.rand(4096,4096)
B = torch.rand_like(A)

In [3]:
# Compute C = AB matrix-matrix multiplication
C = torch.mm(A,B)
print(C)
C.shape

tensor([[1024.8134, 1043.0073, 1023.7675,  ..., 1033.6628, 1022.3378,
         1027.2826],
        [1015.9437, 1022.1133,  997.8547,  ..., 1021.9542, 1015.9304,
         1015.2767],
        [1028.1575, 1030.0774, 1010.2744,  ..., 1026.7228, 1016.0489,
         1021.8556],
        ...,
        [1016.6935, 1022.4557,  998.3668,  ..., 1026.2113, 1017.0294,
         1014.2944],
        [1003.5803, 1013.4774,  982.1767,  ..., 1006.7241,  989.4167,
          990.8864],
        [1015.0729, 1028.7634,  997.0306,  ..., 1030.0721, 1025.7074,
         1023.0061]])


torch.Size([4096, 4096])

In [5]:
# Compute C = AB, treating A as a matrix but computing the result for each column of B one at a time. Report the time.
C = A * B
print(C)

tensor([[0.2175, 0.3455, 0.2794,  ..., 0.0751, 0.4192, 0.1279],
        [0.1015, 0.0862, 0.2217,  ..., 0.5084, 0.2679, 0.2547],
        [0.2622, 0.0145, 0.0625,  ..., 0.1754, 0.2643, 0.0227],
        ...,
        [0.1717, 0.0457, 0.6406,  ..., 0.2956, 0.1386, 0.4799],
        [0.3730, 0.4364, 0.0450,  ..., 0.0055, 0.0497, 0.6473],
        [0.5484, 0.0771, 0.0623,  ..., 0.3378, 0.3878, 0.3050]])


In [6]:
# Compute C = AB, treating A and B as collections of vectors

C = torch.empty(4096, 4096)

#for i in range(4096):
#  for j in range(4096):
#    C[i,j] = torch.dot(A[i, :], B[:, j])
# never run it again cuz it takes forever

C
# C = torch.sum(A_reshaped * B_reshaped, dim = 2)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries.

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix.
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

## 4. NDArray and NumPy

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray.
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [9]:
# compute the norm one at a time and assign its outcome to the np.ndarray
A = np.random.randn(4096,4096)
B = np.random.randn(4096,4096)

c = np.empty(4096)

for i in range(4096):
  d = np.dot(A, B[:,i])
  c[i] = np.power(np.linalg.norm(d), 2)

c

array([16158876.67086491, 16390859.91058355, 17272704.82060322, ...,
       16790162.20973532, 17147823.40055027, 16909854.45794852])

In [10]:
# use a torch.tensor object as an intermediate storage and copy to np.ndarray object at the end
A = torch.rand(4096,4096)
B = torch.rand(4096,4096)

d = torch.empty(4096)

for i in range(4096):
  f = torch.mv(A, B[:, i])
  d[i] = np.power(torch.norm(f), 2)

c = d.numpy()
c

array([4.3437932e+09, 4.3491343e+09, 4.3092475e+09, ..., 4.3677455e+09,
       4.2696269e+09, 4.2702579e+09], dtype=float32)

## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.